In [6]:
#Data import

##Import librairies
import requests
!conda install -c conda-forge folium=0.5.0 --yes
import folium 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import nltk
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
##Get wikipedia data
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url,"lxml")
My_table = soup.find("table",{"class":"wikitable sortable"})

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [7]:
#Build DataFrame

##Create DataFrame from 3 first columns
PostalCode = []
Borough = []
Neighborhood = []
df = pd.DataFrame()

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 1:
        A = cells[0].find(text=True)
        B = cells[1].find(text=True)
        C = cells[2].find(text=True)
        PostalCode.append(A)
        Borough.append(B)
        Neighborhood.append(C)
        
df['PostalCode'] = PostalCode
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
##Data Cleaning
###Retrieve "Not assigned" Borough
df = df[df['Borough'] != "Not assigned"]

###Replace "Not assigned" Neighborhood
for i in range(len(df['Borough'])):
    if df['Neighborhood'].iloc[i] == "Not assigned":
        df['Neighborhood'].iloc[i] = df['Borough'] .iloc[i] 
        
        
df = df.sort_values(by=['PostalCode'])

m = len(df['PostalCode'])
df2 = df

###Save number of distinct value of Postal Code
n = df['PostalCode'].nunique()

In [9]:
###Remove duplicates and concatenate Neighborhood
a = df['PostalCode'].iloc[0]
b = df['Borough'].iloc[0] 
c = df['Neighborhood'].iloc[0]

for i in range(m-1):
    i = i + 1
    if df['PostalCode'].iloc[i] != a:
        df2['PostalCode'].iloc[i]  = df['PostalCode'].iloc[i] 
        df2['Borough'].iloc[i]  = df['Borough'].iloc[i] 
        df2['Neighborhood'].iloc[i]  = ', '.join(set(list(df[df['PostalCode'] == df['PostalCode'].iloc[i]]['Neighborhood'])))
        a = df['PostalCode'].iloc[i]
    else:
        a = df['PostalCode'].iloc[i]
        df2['PostalCode'].iloc[i]  = "Not assigned"
        df2['Borough'].iloc[i]  = "Not assigned"
        df2['Neighborhood'].iloc[i]  = "Not assigned"

In [10]:
###Clean duplicates
df2 = df2[df2['Borough'] != "Not assigned"]

In [11]:
##Sanity checks
###Sanity check df2 size is equal to distinct value of Postal Code
n == len(df2['Borough'] )

True

In [12]:
###Sanity check on example "M5A"
df2[df2['PostalCode'] == "M5A"]

,PostalCode,Borough,Neighborhood
5,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [13]:
###Check the shape of the DataFrame
df2.shape

(103, 3)

In [14]:
#Add latitue/longitude to DataFrame
geo = pd.read_csv('http://cocl.us/Geospatial_data')

In [15]:
df_final =df2.merge(geo, right_on = 'Postal Code' , left_on = 'PostalCode' ,how='left')
df_final.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Rouge,M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood\n, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,M1H,43.773136,-79.239476


In [16]:
#Clustering

##Look at word frequency in Borough
a =[]
for i in range(52):
    b = df2['Borough'].iloc[i].split()
    for b in df2['Borough'].iloc[i].split():
        a.append(b)

nltk.FreqDist(a)

### * We can perform clustering for Toronto, New York, Scarborough

FreqDist({'Central': 6,
          'Downtown': 2,
          'East': 9,
          'North': 18,
          'Scarborough': 17,
          'Toronto': 12,
          'York': 23})

In [17]:
#Toronto

df_Toronto = df_final[df_final['Borough'].str.contains('Toronto')]

##Look at word frequency in Neighborhood
a =[]
for i in range(len(df_Toronto['Neighborhood'])):
    b = df_Toronto['Neighborhood'].iloc[i].split()
    for b in df_Toronto['Neighborhood'].iloc[i].split():
        a.append(b)
nltk.FreqDist(a).most_common(10)

### * We can set a k value under 6 based on neighborhoud word occurences

[(',', 13),
 ('West', 6),
 ('The', 6),
 ('Park', 5),
 ('North', 4),
 ('Toronto', 4),
 ('South', 3),
 ('Hill', 3),
 ('Forest', 3),
 ('Parkdale', 2)]

In [18]:
## K clustering
Features = ['Latitude','Longitude']
kmeans = KMeans(n_clusters= 6, random_state=0).fit(df_Toronto[Features],df_Toronto['Borough'])
df_Toronto['clusters']=kmeans.labels_
kmeans.labels_

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([3, 5, 3, 5, 0, 0, 0, 0, 0, 0, 5, 5, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       0, 4, 4, 4, 2, 2, 2, 4, 1, 4, 4, 1, 1, 1, 3], dtype=int32)

In [19]:
## Mapping

local_map = folium.Map(location=[43.654438,-79.380699], zoom_start=14)

for lat, lon, nei, cluster in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighborhood'],df_Toronto['clusters']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,fill=True,fill_opacity=0.7).add_to(local_map)
   
local_map

In [20]:
#SCARBOROUGH

df_SCARBOROUGH = df_final[df_final['Borough'].str.contains('Scarborough')]

##Look at word frequency in Neighborhood
a =[]
for i in range(len(df_SCARBOROUGH['Neighborhood'])):
    b = df_SCARBOROUGH['Neighborhood'].iloc[i].split()
    for b in df_SCARBOROUGH['Neighborhood'].iloc[i].split():
        a.append(b)
nltk.FreqDist(a).most_common(10)

[(',', 7),
 ('West', 4),
 ('Rouge', 3),
 ('Scarborough', 3),
 ('East', 3),
 ('Park', 3),
 ("L'Amoreaux", 2),
 ('Wexford', 2),
 ('Agincourt', 2),
 ('Village', 2)]

In [21]:
### * We can set a k value under 6 based on neighborhoud word occurences
## K clustering
Features = ['Latitude','Longitude']
kmeans = KMeans(n_clusters= 6, random_state=0).fit(df_SCARBOROUGH[Features],df_final['Borough'])
df_SCARBOROUGH['clusters']=kmeans.labels_
kmeans.labels_

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([2, 5, 5, 3, 3, 3, 1, 1, 1, 1, 4, 4, 0, 0, 0, 0, 2], dtype=int32)

In [22]:
## Mapping

local_map = folium.Map(location=[43.774438,-79.233238], zoom_start=12)
markers_colors = []
for lat, lon, nei, cluster in zip(df_SCARBOROUGH['Latitude'], df_SCARBOROUGH['Longitude'], df_SCARBOROUGH['Neighborhood'],df_SCARBOROUGH['clusters']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],radius=5,popup=label,fill=True,fill_opacity=0.7).add_to(local_map)
   
local_map